In [2]:
pip install rasterio matplotlib Pillow folium

     --------------------------------------- 25.4/25.4 MB 10.7 MB/s eta 0:00:00
     ---------------------------------------- 110.9/110.9 KB ? eta 0:00:00
     ---------------------------------------- 98.2/98.2 KB ? eta 0:00:00
     ---------------------------------------- 63.8/63.8 KB ? eta 0:00:00
     ---------------------------------------- 90.4/90.4 KB 5.3 MB/s eta 0:00:00
     ---------------------------------------- 64.9/64.9 KB ? eta 0:00:00
     -------------------------------------- 134.9/134.9 KB 7.8 MB/s eta 0:00:00
     ---------------------------------------- 70.4/70.4 KB 4.0 MB/s eta 0:00:00
     ---------------------------------------- 128.7/128.7 KB ? eta 0:00:00
     -------------------------------------- 105.8/105.8 KB 6.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\dohya\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import geopandas as gpd
import folium
import pandas as pd

# 1. Shapefile 불러오기
jejusi = gpd.read_file("Data/제주시.shp")
seogwipo = gpd.read_file("Data/서귀포시.shp")

# 2. 좌표계를 WGS84 (EPSG:4326)로 변환
jejusi = jejusi.to_crs(epsg=4326)
seogwipo = seogwipo.to_crs(epsg=4326)

# 3. 두 GeoDataFrame 병합
jeju_all = gpd.GeoDataFrame(pd.concat([jejusi, seogwipo], ignore_index=True), crs=jejusi.crs)

# 4. 중심 좌표 계산
center = jeju_all.geometry.union_all().centroid  # 또는 union_all().centroid (최신 버전용)

# 5. folium 지도 생성
m = folium.Map(location=[center.y, center.x], zoom_start=11, tiles="OpenStreetMap")

# 6. GeoJson 레이어 추가
folium.GeoJson(
    jeju_all,
    name="제주 임도 (제주시 + 서귀포시)",
    style_function=lambda feature: {
        'color': 'red',
        'weight': 2
    },
    tooltip=folium.GeoJsonTooltip(fields=jeju_all.columns[:2].tolist(), aliases=["속성1", "속성2"])
).add_to(m)

# 7. 레이어 컨트롤
folium.LayerControl().add_to(m)

# 8. 지도 출력
m


In [1]:
# 📦 필요한 패키지 설치 (colab일 경우)
# !pip install geopandas rasterio folium shapely

import os
import geopandas as gpd
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape
import folium

# 1️⃣ GeoTIFF 경로 설정 (경로에 맞게 수정)
tif_path = "Data/50.tif"



---------------------------

In [2]:
# 2️⃣ TIF 래스터 → 벡터화 (1~5 위험등급 polygon 추출)
with rasterio.open(tif_path) as src:
    image = src.read(1)  # 1-band
    mask = image >= 1    # 위험등급 1~5 범위만 추출
    transform = src.transform
    crs = src.crs

    results = (
        {'properties': {'value': int(v)}, 'geometry': s}
        for s, v in shapes(image, mask=mask, transform=transform)
    )
    geoms = list(results)


In [3]:
# 3️⃣ GeoDataFrame으로 변환
gdf = gpd.GeoDataFrame.from_features(geoms)
gdf.set_crs(crs, inplace=True)       # 원래 좌표계 설정
gdf = gdf.to_crs(epsg=4326)          # 위도/경도 좌표계로 변환

In [4]:
# 4️⃣ folium 스타일 함수 정의 (등급별 색상 지정)
def style_function(feature):
    value = feature['properties']['value']
    color = {
        1: '#ffffcc',  # 매우 낮음
        2: '#ffeda0',  # 낮음
        3: '#feb24c',  # 보통
        4: '#fd8d3c',  # 높음
        5: '#e31a1c'   # 매우 높음
    }.get(value, 'gray')
    return {
        'fillColor': color,
        'color': color,
        'weight': 0.5,
        'fillOpacity': 0.6
    }

In [5]:
# 5️⃣ 중심 좌표 계산
center = gdf.geometry.union_all().centroid

In [6]:
# 6️⃣ folium 지도 생성
m = folium.Map(location=[center.y, center.x], zoom_start=11)

In [ ]:
# 7️⃣ GeoJson 벡터 레이어 추가
folium.GeoJson(
    gdf,
    name="산사태 위험도 벡터",
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=["value"], aliases=["위험등급"])
).add_to(m)

folium.LayerControl().add_to(m)

m 
